# LangGraph Chatbot
This notebook wraps a LangGraph `StateGraph` in a conversational loop using Google Gemini‑2.0‑Flash as the LLM.
It will:
1. Maintain a message history state.
2. Invoke the LLM to generate replies.
3. Print a chat‑style interaction.

In [5]:
from typing import TypedDict, Annotated
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Load environment variables (e.g., LLM credentials) from a .env file
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# Instantiate the LLM (Google Gemini 2.0 Flash) via LangChain
llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    """
    Conversation state schema:
    - messages: list of dicts with 'role' and 'content' for LLM context
    """
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    """
    Node that invokes LLM on state["messages"] and returns a new state.
    """
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)

# Add a single node named "chatbot_node" running our chatbot() function
builder.add_node("chatbot_node", chatbot)

# Wire START → chatbot_node → END
builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

# Compile into an executable StateGraph
graph = builder.compile()

In [7]:
# Test the graph once: ask a simple factual question
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
response = graph.invoke({"messages": [message]})

response['messages']

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='afa5e80a-3fd0-4d93-a284-6084b1a7eac8'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--17239452-11de-4ec1-ac3a-dfc21d2c70c0-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

## Interactive Chat Loop
Start a REPL to chat continuously.
Type `quit` or `exit` to end the session.

In [8]:
# Begin an interactive chat loop (type "quit" or "exit" to stop)
state = None
while True:
    # Read user input
    in_message = input("You: ")
    print("Human:", in_message)
    if in_message.lower() in {"quit", "exit"}:
        break

    # Initialize state on the first message
    if state is None:
        state: State = {
            "messages" : [{"role": "user", "content": in_message}]
        }
    else:
        # Append each new user message
        state["messages"].append({"role":"user", "content": in_message})

    # Invoke the graph to get the assistant’s reply
    state = graph.invoke(state)
    # Print out the assistant’s latest response
    print("Bot:", state["messages"][-1].content)

Human: Who walked on the moon for the first time?
Bot: Neil Armstrong was the first person to walk on the moon.
Human: What is the current stock price of Amazon?
Bot: I am unable to provide real-time stock prices. Stock prices change constantly.

To get the current stock price of Amazon (AMZN), I recommend checking a reliable source such as:

*   **Google Finance:** Search "Amazon stock price" on Google.
*   **Yahoo Finance:** finance.yahoo.com
*   **Bloomberg:** bloomberg.com
*   **Your Brokerage Account:** If you have a brokerage account, the price will be displayed there.
Human: quit
